In [ ]:
pip install folium

In [43]:
pip install pyproj

     |████████████████████████████████| 6.3 MB 5.4 MB/s 


In [44]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
import os
import numpy as np

import folium
from folium import plugins
print(folium.__version__)

import pandas as pd
import numpy as np
import pyproj
import folium

0.8.3


# 1. 인허가 정보 데이터 불러오기
* 사용할 데이터
  * 의원 인허가 정보
  * 

In [45]:
def project_array(coord, p1_type, p2_type):
    """
    좌표계 변환 함수
    - coord: x, y 좌표 정보가 담긴 NumPy Array
    - p1_type: 입력 좌표계 정보 ex) epsg:5179
    - p2_type: 출력 좌표계 정보 ex) epsg:4326
    """
    p1 = pyproj.Proj(init=p1_type)
    p2 = pyproj.Proj(init=p2_type)
    fx, fy = pyproj.transform(p1, p2, coord[:, 0], coord[:, 1])
    return np.dstack([fx, fy])[0]

In [73]:
# 합쳐둘 데이터프레임
data = pd.DataFrame({'업태구분명':[], '좌표정보(X)':[], '좌표정보(Y)':[], '위도':[], '경도':[]})

In [74]:
# 모든 데이터 동일하게 합치기
file_list = os.listdir('/content/drive/MyDrive/dacon/GSBC/map')
for df in file_list:
  name = df.split(' ')[2]
  df = pd.read_csv('/content/drive/MyDrive/dacon/GSBC/map/'+df, encoding='cp949') # 데이터 불러오기
  df = df[df['상세영업상태명']!='폐업'] # 폐업매장 제외
  df = df[['좌표정보(X)', '좌표정보(Y)']].dropna() # 결측치 제거

  df_np = np.array(df)

  # 좌표계 정보 설정
  p1_type = "epsg:2097"
  p2_type = "epsg:4326"

  # project_array() 함수 실행
  result = project_array(df_np, p1_type, p2_type)

  df['업태구분명'] = [name]*len(df)
  df['경도'] = result[:, 0]
  df['위도'] = result[:, 1]
  data = pd.concat([data, df], axis=0)

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  # Remove the CWD f

In [75]:
data

,업태구분명,좌표정보(X),좌표정보(Y),위도,경도
1,의원,185701.539545,447825.366608,37.532563,126.836129
8,의원,186003.507038,448917.760317,37.542410,126.839524
9,의원,186347.811013,447862.079517,37.532904,126.843440
11,의원,187762.739786,447807.356880,37.532430,126.859449
12,의원,186060.144036,449071.063990,37.543792,126.840162
...,...,...,...,...,...
160,안경업,185163.000000,450826.000000,37.559590,126.829975
161,안경업,182524.823836,451438.250897,37.565060,126.800102
162,안경업,184503.000000,451852.000000,37.568823,126.822483
163,안경업,186501.233193,451485.251876,37.565550,126.845108


# 2. 지도에 맵핑하기

In [50]:
# 데이터 100개 랜덤 추출
sample = hospital.sample(n=100)

# 지도 중심 좌표 설정
lat_c, lon_c = 37.53165351203043, 126.9974246490573

# Folium 지도 객체 생성
m = folium.Map(location=[lat_c, lon_c], zoom_start=6)

# 마커 생성
for _, row in sample.iterrows():
    lat, lon = row['위도'], row['경도']
    folium.Marker(location=[lat, lon]).add_to(m)
  
m